In [1]:
import zipfile
import random
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import save_image
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.utils import save_image
import matplotlib.pyplot as plt



### Load data

In [4]:
import zipfile

zip_path = "minefree-class-split-wo-borders.zip"

extract_to = "data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete!")


Extraction complete!


### Diffusion models training


In [6]:
!pip install denoising-diffusion-pytorch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
data_path = "data/minefree-class-split-wo-borders/train"

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
train_dataset = datasets.ImageFolder(root=data_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

from collections import Counter
labels = [label for _, label in train_dataset]
print("Class distribution:", Counter(labels))


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classes: ['bombed', 'not bombed']
Class distribution: Counter({0: 402, 1: 68})


In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.transforms import InterpolationMode

image_size = 64
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((image_size, image_size), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = datasets.ImageFolder(root="data/minefree-class-split-wo-borders/train", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

num_classes = len(dataset.classes)


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleUNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.embed_class = nn.Embedding(num_classes, 64)
        self.conv1 = nn.Conv2d(3 + 1, 64, 3, padding=1)  # 3 channels + 1 class channel
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.out = nn.Conv2d(64, 3, 1)

    def forward(self, x, class_labels):
        class_embed = self.embed_class(class_labels).view(-1, 1, 1, 64)
        class_embed = class_embed.repeat(1, x.shape[2], x.shape[3], 1).permute(0, 3, 1, 2)
        x = torch.cat([x, class_embed[:, :1, :, :]], dim=1)  # Add 1 channel
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out(x)


In [14]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.utils import save_image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_size = (3, 64, 64)
batch_size = 16
epochs = 500
T = 1000

betas = torch.linspace(1e-4, 0.02, T).to(device)
alphas = 1. - betas
alpha_bars = torch.cumprod(alphas, dim=0).to(device)

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

data_path = "data/minefree-class-split-wo-borders/train"
dataset = datasets.ImageFolder(data_path, transform=transform)

bombed_indices = [i for i, (_, label) in enumerate(dataset.samples) if label == 0]
not_bombed_indices = [i for i, (_, label) in enumerate(dataset.samples) if label == 1]

bombed_dataset = Subset(dataset, bombed_indices)
not_bombed_dataset = Subset(dataset, not_bombed_indices)

bombed_loader = DataLoader(bombed_dataset, batch_size=batch_size, shuffle=True)
not_bombed_loader = DataLoader(not_bombed_dataset, batch_size=batch_size, shuffle=True)

class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.down1 = nn.Conv2d(3, 64, 3, padding=1)
        self.down2 = nn.Conv2d(64, 128, 3, padding=1)
        self.up1 = nn.ConvTranspose2d(128, 64, 3, padding=1)
        self.out = nn.Conv2d(64, 3, 1)

    def forward(self, x, t):
        x = F.relu(self.down1(x))
        x = F.relu(self.down2(x))
        x = F.relu(self.up1(x))
        return self.out(x)

def train_diffusion(model, dataloader, name="model"):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(epochs):
        for images, _ in dataloader:
            images = images.to(device)
            optimizer.zero_grad()

            t = torch.randint(0, T, (images.size(0),), device=device)
            alpha_bar_t = alpha_bars[t].view(-1, 1, 1, 1)

            noise = torch.randn_like(images)
            noisy_images = torch.sqrt(alpha_bar_t) * images + torch.sqrt(1 - alpha_bar_t) * noise

            predicted_noise = model(noisy_images, t)
            loss = F.mse_loss(predicted_noise, noise)

            loss.backward()
            optimizer.step()

        print(f"[{name}] Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    return model

@torch.no_grad()
def sample(model, img_size=(3, 64, 64), steps=T):
    model.eval()
    x = torch.randn((1, *img_size)).to(device)
    for t in reversed(range(steps)):
        t_batch = torch.full((1,), t, device=device, dtype=torch.long)
        beta_t = betas[t]
        alpha_t = alphas[t]
        alpha_bar_t = alpha_bars[t]

        predicted_noise = model(x, t_batch)
        noise = torch.randn_like(x) if t > 0 else 0

        x = (1 / torch.sqrt(alpha_t)) * (x - ((1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)) * predicted_noise) + torch.sqrt(beta_t) * noise
    return x.clamp(0, 1)

print("Training model for bombed")
model_bombed = train_diffusion(SimpleUNet(), bombed_loader, name="BOMBED")

print("Training model for not bombed")
model_not_bombed = train_diffusion(SimpleUNet(), not_bombed_loader, name="NOT_BOMBED")

sample_bombed = sample(model_bombed)
sample_not_bombed = sample(model_not_bombed)

save_image(sample_bombed, "sample_bombed.png")
save_image(sample_not_bombed, "sample_not_bombed.png")


Training BOMBDED model...
[BOMBED] Epoch 1/500, Loss: 0.7476
[BOMBED] Epoch 2/500, Loss: 0.6238
[BOMBED] Epoch 3/500, Loss: 0.1249
[BOMBED] Epoch 4/500, Loss: 0.0519
[BOMBED] Epoch 5/500, Loss: 0.0466
[BOMBED] Epoch 6/500, Loss: 0.0350
[BOMBED] Epoch 7/500, Loss: 0.1985
[BOMBED] Epoch 8/500, Loss: 0.0418
[BOMBED] Epoch 9/500, Loss: 0.0412
[BOMBED] Epoch 10/500, Loss: 0.0416
[BOMBED] Epoch 11/500, Loss: 0.1887
[BOMBED] Epoch 12/500, Loss: 0.0429
[BOMBED] Epoch 13/500, Loss: 0.0408
[BOMBED] Epoch 14/500, Loss: 0.0368
[BOMBED] Epoch 15/500, Loss: 0.0412
[BOMBED] Epoch 16/500, Loss: 0.3014
[BOMBED] Epoch 17/500, Loss: 0.0571
[BOMBED] Epoch 18/500, Loss: 0.0356
[BOMBED] Epoch 19/500, Loss: 0.1113
[BOMBED] Epoch 20/500, Loss: 0.2317
[BOMBED] Epoch 21/500, Loss: 0.0334
[BOMBED] Epoch 22/500, Loss: 0.0328
[BOMBED] Epoch 23/500, Loss: 0.0314
[BOMBED] Epoch 24/500, Loss: 0.0310
[BOMBED] Epoch 25/500, Loss: 0.0279
[BOMBED] Epoch 26/500, Loss: 0.0441
[BOMBED] Epoch 27/500, Loss: 0.0308
[BOMBED] Ep